Problem 1
======================

In [94]:
import numpy as np
import scipy as sp

In [89]:
def integrate(g, bounds, N, method ):
    a, b = bounds

    if method == "midpoint":
        x = np.empty((N))
        y = np.empty((N))
       
        for i in range (0, N):
            x[i] = a + (((2 * i + 1) * (b - a)) / (2 * N))
        y = g(x)
        value = ((b - a) / N) * np.sum(y[0:N-1])
    
    if method == "trapezoid":
        x = np.empty((N + 1))
        y = np.empty((N + 1))
        for i in range (0, N + 1):
            x[i] = a + ((i * (b - a)) /(N))
        y = g(x)
        value =  ( (b - a) / (2 * N)) * (g(x[0]) + 2 * np.sum(y[1:N - 1]) + g(x[N]))

    if method == "Simpsons":
        x = np.empty((2 * N + 1))
        y = np.empty((2 * N + 1))
        for i in range (0, 2 * N + 1):
            x[i] = a + ((i * (b - a)) / (2 * N))
        y = g(x)
        y0 = y[0]
        y = y[1:]
        y = y.reshape(int(N), 2)
        yodd, yeven = np.hsplit(y, 2)
        value = ( (b - a) / (3 * (2 * N + 1)) ) * \
        (y0 + 2 * np.sum(yeven[:(N-2)]) + 4 * np.sum(yodd) + yeven[N-1])
    
    return value

def g(x):
    return 0.1 * x ** 4 - 1.5 * x ** 3 + 0.53 * x ** 2 + 2 * x + 1

bounds = np.array((-10,10))
N = 10000
value_mid = integrate(g, bounds, N, 'midpoint')
value_trap = integrate(g, bounds, N, 'trapezoid')
value_simp = integrate(g, bounds, N, 'Simpsons')

print("Value for Midpoint Method:", value_mid)
print("Value for Midpoint Method:", value_trap)
print("Value for Midpoint Method:", value_simp)

Value for Midpoint Method: 4374.18512164
Value for Midpoint Method: 4374.18545734
Value for Midpoint Method: [ 4373.39861349]


Problem 2
======================

In [157]:
from scipy.stats import norm

def newton(mu, s, N, k):
    
    Z = np.linspace( mu - k * s, mu + k * s, N)
    weights = np.empty_like(Z)
    
    for j in range (0, N):
        
        if j == 0:
            weights[j] = norm.cdf((Z[0] + Z[1]) / 2, mu, s)
        
        elif j == N - 1:
            w = 1 - norm.cdf((Z[N - 1] + Z[N - 2]) / 2, mu, s)
            if w == np.inf:
                weights[j] = 0
            else:
                weights[j] = w
        else:
            weights[j] = norm.cdf((Z[j] + Z[j + 1]) / 2, mu, s) - norm.cdf((Z[j - 1] + Z[j]) / 2, mu, s)
        
    return Z, weights 

Z, weights = newton(0, 1, 11, 3)

print(Z, weights)
print(np.sum(weights))


[-3.  -2.4 -1.8 -1.2 -0.6  0.   0.6  1.2  1.8  2.4  3. ] [ 0.00346697  0.01439745  0.04894278  0.11725292  0.19802845  0.23582284
  0.19802845  0.11725292  0.04894278  0.01439745  0.00346697]
1.0
(11,)


Problem 3
======================

In [179]:
def lognormal(mu, s, N, k):

    Z, weights = newton(mu, s, N, k)
    
    approximation = weights.reshape(1,N) @ (np.exp(Z).reshape(N,1))
    
    return approximation

approx = lognormal(0, 1, 11, 3)
print(approx)


[[ 1.66394095]]


Problem 4
======================

In [197]:
def lognormal_mean(mu, s, N, k):

    Z, weights = newton(mu, s, N, k)
    
    approximation = weights.reshape(1,N) @ np.multiply((np.exp(Z).reshape(N,1)), Z.reshape(N,1))
    
    return approximation

mean = lognormal(10.5, 0.8, 1000, 6)

print("My mean: ", mean)

print("Exact value: ", np.exp(10.5 + ((0.8 ** 2) / 2)))

My mean:  [[ 50011.27880633]]
Exact value:  50011.0870085


Problem 5
======================

In [233]:
def g(x):
    return 0.1 * x ** 4 - 1.5 * x ** 3 + 0.53 * x ** 2 + 2 * x + 1

bounds = np.array((-10,10))
a, b = bounds 

x0 = lambda x: 1
constant = sp.integrate.quad(x0, a, b)[0]
x1 = lambda x: x
first = sp.integrate.quad(x1, a, b)[0]
x2 = lambda x: x**2
second = sp.integrate.quad(x2, a, b)[0]
x3 = lambda x: x**3
third = sp.integrate.quad(x3, a, b)[0]
fourth = sp.integrate.quad(x4, a, b)[0]
x5 = lambda x: x**5
fifth = sp.integrate.quad(x5, a, b)[0]

initial = np.random.rand((6))
#w0, w1, w2, x0, x1, x2 = initial

def system(s):
    return [ s[0] + s[1] + s[2] - constant, s[0]*s[3] + s[1] *s[4] + s[2]*s[5] - first,\
            s[0]*s[3]**2 + s[1] *s[4]**2 + s[2]*s[5]**2 - second,\
           s[0]*s[3]**3 + s[1] *s[4]**3 + s[2]*s[5]**3 - third,\
           s[0]*s[3]**4 + s[1] *s[4]**4 + s[2]*s[5]**4 - fourth,\
           s[0]*s[3]**5 + s[1] *s[4]**5 + s[2]*s[5]**5 - fifth]

sol = sp.optimize.root(system, initial)

w0, w1, w2, x0, x1, x2 = sol.x

print(sol.x)

w = sol.x[:3]
z = sol.x[3:]

integral = w @ (g(z)).reshape(3,1)
print(integral)

[  8.88888889e+00   5.55555556e+00   5.55555555e+00  -2.03170561e-09
  -7.74596669e+00   7.74596669e+00]
[ 4373.33333379]


Problem 6
======================

In [235]:
value = sp.integrate.quad(g, a, b)
print(value[0])

4373.333333333334


Scipy's function is exact, and the Gaussian is close, but the other methods are a little further away. 

Problem 7
======================

In [255]:
from numba import jit
import inspect
from numpy import interp

@jit
def g7(points):
    x, y = points
    v = np.empty_like(x)
    e = x ** 2 + y ** 2
    
    for i in range (0, len(e)):
    
        if e[i] <= 1:
            v[i] = 1.0
        else:
            v[i] = 0.0
    return v
    

def pi(g7, dom, N):
    
    xl, xu, yl, yu = dom
    
    x = np.random.uniform(xl, xu, N)
    y = np.random.uniform(yl, yu, N)
    points = np.vstack((x, y))
    
    values = g7(points)
    
    pi = (((yu - yl) * (xu - xl)) / N ) * np.sum(values)
    
    return pi

print("pi = ", pi(g7, (-1, 1, -1, 1), 30000000))

pi =  3.1415912


Took roughly 30 million iterations

Problem 8
======================

In [4]:
import sympy as sp
import math as m
import numpy as np

def Niederreiter(n, s):
    seq = np.zeros((s, n))
    for i in range(1, n + 1):
        for j in range(1, s + 1):
            seq[j - 1, i - 1] = ((i) * 2. ** ((j) / (j + 1.)) -
                                 m.floor((i) * 2. **
                                 ((j) / (j + 1.))))
    return seq

def Baker(n, s):
    seq = np.zeros((s,n))
    for i in range(1, n+1):
        for j in range(1, s+1):
            seq[j-1, i-1] = ((i+1)*np.exp(j + 1)) - m.floor((i + 1) *
                    np.exp(j + 1))
    return seq

def sample(n, d, seq):
    
    p = sp.prime(d)
    
    if seq == 'weyl':
        element = (n * p ** 0.5) - m.floor(n * p ** 0.5)
        
    elif seq == 'haber':
        element = (((n * (n+1)) / 2) * p ** 0.5) - m.floor(((n * (n+1)) / 2) * p ** 0.5)
        
    elif seq == 'nied':
        #element = (n * 2 ** ( d / (n + 1))) - m.floor(n * 2 ** ( d / (n + 1)))
        element = Niederreiter(n, d)
        
    elif seq == 'baker':
        element = Baker(n,d)
        #element = (n * np.exp(p)) - m.floor(n * np.exp(p))
    
    else:
        print ('Sequence type does not exist')
    
    return element

print(sample(2, 2, 'weyl'))  
print(sample(2, 2, 'haber'))  
print(sample(2, 2, 'nied'))  
print(sample(2, 2, 'baker'))  

0.4641016151377544
0.19615242270663202
[[ 0.41421356  0.82842712]
 [ 0.58740105  0.1748021 ]]
[[ 0.7781122   0.1671683 ]
 [ 0.17107385  0.25661077]]


Problem 9
===================

In [309]:
@jit
def g9(points):
    x, y = points
    v = np.empty_like(x)
    e = x ** 2 + y ** 2
    
    for i in range (0, len(e)):
    
        if e[i] <= 1:
            v[i] = 1.0
        else:
            v[i] = 0.0
    return v

@jit
def loop(x, y, N):
    
    for i in range (0, N):
        x[i] = sample(i + 1, 1, 'haber')
        y[i] = sample(i + 1, 1, 'weyl')
    
    return x, y

@jit
def pi9(g7, dom, N):
    
    xl, xu, yl, yu = dom
    
    x = np.empty((N))
    y = np.empty((N))
    
    x, y = loop(x, y, N)
    
    points = np.vstack((x, y))
    values = g7(points)
    
    pi = (((yu - yl) * (xu - xl)) / N ) * np.sum(values)
    
    return pi

print("pi = ", pi9(g9, (-1, 1, -1, 1), 2190000))

pi =  3.14159452055


It converged with roughly 2.2 million iterations.